## Notebook 4

Binomial proportions. In the latter half of the dataset there are genotypic counts. This notebook tries to model the proportion between coluzzii and gambiae, and see if there is any variation among villages and months. 

The subset we are using are counts from 2017-2019, wet season only (25% of the dataset). 

In [22]:
# LOAD R PACKAGES
require(compiler)
enableJIT(3)
setMKLthreads(22)
require(lme4)
require(lmerTest)
require(MASS)

# LOAD DATASET dat1
#setwd('variance/Florian')
load('Per house data PSC 2012 to 2019 polish2.RData')
load('BF_weather.RData')
ls()

[1] 3

[1] "BV_weather"       "col.f.proportion" "dat1"             "m"               
 [5] "m_0"              "m_1"              "m1"               "m2"              
 [9] "overdispersion"   "PL_weather"       "SK_weather"

In [23]:
# SUBSET (2017-2019)
dat1<-dat1[dat1$year.assigned %in% 2017:2019,]
# SUBSET (WET SEASON MAY - OCT)
dat1<-dat1[dat1$month.assigned %in% 5:10,]
dim(dat1)
names(dat1)
# TRANSFORM num.persons
persons.status<-dat1$num.persons
persons.status[dat1$num.persons>3]<-'Hi'
persons.status[dat1$num.persons<=3]<-'Low'
persons.status[dat1$num.persons==0]<-'None'

[1] 876  26

[1] "village"            "month.assigned"     "year.assigned"     
 [4] "monthyear.assigned" "day"                "month"             
 [7] "year"               "date.collection"    "site.id"           
[10] "longitude"          "latitude"           "room.id"           
[13] "num.persons"        "num.persons.remark" "mosquito.net"      
[16] "permanence"         "count.m"            "genotyped.m"       
[19] "col.m"              "gam.m"              "arab.m"            
[22] "count.f"            "genotyped.f"        "col.f"             
[25] "gam.f"              "arab.f"

In [24]:
# DESCRIPTIVE STATS ON THE PROPORTIONS (VERY ROUGH)
col.f.proportion<-dat1$col.f/(dat1$col.f+dat1$gam.f)
by(col.f.proportion, dat1$village, mean, na.rm=T)
by(col.f.proportion, dat1$month.assigned, mean, na.rm=T)

dat1$village: Bana market
[1] 0.9157716
------------------------------------------------------------ 
dat1$village: Bana village
[1] 0.9172278
------------------------------------------------------------ 
dat1$village: Pala
[1] 0.2774607
------------------------------------------------------------ 
dat1$village: Souroukoudingan
[1] 0.6235339

dat1$month.assigned: 1
[1] NA
------------------------------------------------------------ 
dat1$month.assigned: 2
[1] NA
------------------------------------------------------------ 
dat1$month.assigned: 3
[1] NA
------------------------------------------------------------ 
dat1$month.assigned: 4
[1] NA
------------------------------------------------------------ 
dat1$month.assigned: 5
[1] 0.9792023
------------------------------------------------------------ 
dat1$month.assigned: 6
[1] 0.6047952
------------------------------------------------------------ 
dat1$month.assigned: 7
[1] 0.8558036
------------------------------------------------------------ 
dat1$month.assigned: 8
[1] 0.6601493
------------------------------------------------------------ 
dat1$month.assigned: 9
[1] 0.9415679
------------------------------------------------------------ 
dat1$month.assigned: 10
[1] 0.717817
------------------------------------------------------------ 
dat1$month.assigned: 11
[1] NA
-------

In [25]:
overdispersion<-1:nrow(dat1)
m_0<-glmer(cbind(col.f, gam.f)~(1|overdispersion)+village*month.assigned, 
         data=dat1, family='binomial', 
         control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
summary(m_0)

fixed-effect model matrix is rank deficient so dropping 6 columns / coefficients

Correlation matrix not shown by default, as p = 18 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: cbind(col.f, gam.f) ~ (1 | overdispersion) + village * month.assigned
   Data: dat1
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
  1252.0   1337.6   -607.0   1214.0      648 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.7353 -0.3886  0.2219  0.5028  2.3862 

Random effects:
 Groups         Name        Variance Std.Dev.
 overdispersion (Intercept) 0.5168   0.7189  
Number of obs: 667, groups:  overdispersion, 667

Fixed effects:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             3.99488    0.42990   9.293  < 2e-16 ***
villageBana village                     1.07057    0.84117   1.273  0.20312    
villagePala                            -3.34558    0.40588  -8.243  < 2e-16 ***
villageSouroukoudin

In [26]:
m_1<-glmer(cbind(col.f, gam.f)~(1|overdispersion)+village*month.assigned+mosquito.net, 
         data=dat1, family='binomial', 
         control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
summary(m_1)

fixed-effect model matrix is rank deficient so dropping 6 columns / coefficients

Correlation matrix not shown by default, as p = 19 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cbind(col.f, gam.f) ~ (1 | overdispersion) + village * month.assigned +  
    mosquito.net
   Data: dat1
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
  1231.6   1321.4   -595.8   1191.6      639 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.6993 -0.3741  0.2246  0.4806  2.3666 

Random effects:
 Groups         Name        Variance Std.Dev.
 overdispersion (Intercept) 0.5422   0.7363  
Number of obs: 659, groups:  overdispersion, 659

Fixed effects:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             3.98691    0.48806   8.169 3.11e-16 ***
villageBana village                     1.07029    0.84318   1.269  0.20432    
villagePala                            -3.49189    0.43127  -8.097 5.64e-16 *

In [27]:
m_2<-glmer(cbind(col.f, gam.f)~(1|overdispersion)+village*month.assigned+persons.status, 
         data=dat1, family='binomial', 
         control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
summary(m_2)

fixed-effect model matrix is rank deficient so dropping 6 columns / coefficients

Correlation matrix not shown by default, as p = 20 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cbind(col.f, gam.f) ~ (1 | overdispersion) + village * month.assigned +  
    persons.status
   Data: dat1
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
  1228.3   1322.4   -593.2   1186.3      632 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.7369 -0.3861  0.2182  0.4784  2.3890 

Random effects:
 Groups         Name        Variance Std.Dev.
 overdispersion (Intercept) 0.53     0.728   
Number of obs: 653, groups:  overdispersion, 653

Fixed effects:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             3.97943    0.44295   8.984  < 2e-16 ***
villageBana village                     1.07423    0.84234   1.275 0.202205    
villagePala                            -3.48540    0.42773  -8.149 3.68e-16

Persons and mosquito net does not affect the ratio of coluzzii and gambiae. Different month x village combinations have different ratio. 

Note that we did not genotype all mosquitoes we collected. It is worth thinking how we can combine the two models (Poisson count glm for the combined counts and then the binomial glm for the proportion) for power analysis. 

One simple solution is to use the implied coluzzi and gambiae counts (count.f * proportion). This is quite good when genotyped.f is similar to count.f. 

Another method is to build a complex hierachical model. The observed female counts follow a Poisson distribution, and that the observed female coluzzii count follows another hypergeometric distribution (sample without replacement). Then we can build a probabilistic model on how many coluzzii there are in the sampled pool. 